In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import sys
import os

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.base import BaseEstimator, TransformerMixin

import mlflow
import mlflow.sklearn
from sklearn.model_selection import ParameterGrid

sns.set(style="darkgrid")

In [4]:
class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, variable):
        self.variable = variable

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for var in self.variable:
            X[var] = pd.to_datetime(X[var])
            X[var + '_year'] = X[var].dt.year
            X[var + '_month'] = X[var].dt.month
            X[var + '_day'] = X[var].dt.day
            X[var + '_weekday'] = X[var].dt.dayofweek
            
            X.drop(columns=[var], inplace=True)
        return X

# Cargar Datos
df = pd.read_excel('../data/raw/data_sales_forecasting.xlsx', sheet_name='Base de Datos')

# Ordenar por fecha
df['Fecha'] = pd.to_datetime(df['Fecha'])
df.sort_values('Fecha', inplace=True)
df.reset_index(drop=True, inplace=True)

date_vars = ['Fecha']

cat_vars = [
    'Codigo_Cupon', 
    'Descripcion_Cupon', 
    'Codigo_Producto', 
    'Tipo_Orden', 
    'Tipo_Pago', 
    'Canal_Orden'
]

possible_nums = ['Cantidad_Vendida', 'Precio_Menu_GTQ', 'No_Tienda']
num_vars = [c for c in possible_nums if c in df.columns]

# Construir el Pipeline de Preprocesamiento
preprocessor = ColumnTransformer([
    ('date', Pipeline([('extractor', DateFeatureExtractor(date_vars))]), date_vars),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]), cat_vars),
    ('num', StandardScaler(), num_vars)
], remainder='drop') 

feature_pipeline = Pipeline([('preprocessor', preprocessor)])

print("Pipeline de preprocesamiento creado.")

Pipeline de preprocesamiento creado.


In [5]:
target = 'Venta_Neta_GTQ'
X = df.drop(columns=[target])
y = df[target]

split_point = int(len(df) * 0.80)

X_train = X.iloc[:split_point]
y_train = y.iloc[:split_point]
X_val = X.iloc[split_point:]
y_val = y.iloc[split_point:]

print(f"Train: {X_train.shape[0]} filas | Validación: {X_val.shape[0]} filas")

Train: 521097 filas | Validación: 130275 filas


In [6]:
print("Transformando datos...")
X_train_processed = feature_pipeline.fit_transform(X_train, y_train)
X_val_processed = feature_pipeline.transform(X_val)

print(f"Datos transformados. Nueva forma: {X_train_processed.shape}")

Transformando datos...


c:\Users\gabri\anaconda3\envs\env_papd_project\lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Datos transformados. Nueva forma: (521097, 220)


In [7]:
models_config = {
    'Ridge': {
        'model': Ridge(random_state=2025),
        'params': {'alpha': [0.1, 1.0, 10.0]}
    },
    'Lasso': {
        'model': Lasso(random_state=2025),
        'params': {'alpha': [0.01, 0.1, 1.0]}
    },
    'DecisionTree': {
        'model': DecisionTreeRegressor(random_state=2025),
        'params': {'max_depth': [5, 10, 20]}
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=2025, n_jobs=-1),
        'params': {'n_estimators': [50, 100, 150]}
    },
    'GradientBoosting': {
        'model': GradientBoostingRegressor(random_state=2025),
        'params': {'learning_rate': [0.05, 0.1, 0.2]}
    }
}

In [8]:
# Configuración de MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Sales_Forecasting_Championship_NB")

results = []
best_score = float('inf')
best_model_name = None
best_params = None
best_model_instance = None

print(">>> Iniciando Torneo de Modelos con MLflow <<<")

# Iniciamos el ciclo de entrenamiento completo
with mlflow.start_run(run_name="Notebook_Training_Cycle") as parent_run:
    
    for name, config in models_config.items():
        print(f"\n--> Evaluando Familia: {name}")
        grid = list(ParameterGrid(config['params']))
        
        for params in grid:
            # Nombre para identificar cada prueba
            run_name = f"Challenger_{name}"
            
            # 'nested=True' crea una sub-corrida para cada variante
            with mlflow.start_run(run_name=run_name, nested=True):
                
                # 1. Entrenar
                model = config['model'].set_params(**params)
                model.fit(X_train_processed, y_train)
                
                # 2. Predecir y Evaluar
                preds = model.predict(X_val_processed)
                rmse = np.sqrt(mean_squared_error(y_val, preds))
                
                # 3. Registrar en MLflow 
                mlflow.log_params(params)
                mlflow.log_param("model_type", name)
                mlflow.log_metric("rmse", rmse)
                
                # Etiqueta vital para filtrar después
                mlflow.set_tag("model_role", "Challenger")
                
                results.append({'Model': name, 'Params': str(params), 'RMSE': rmse})
                print(f"    {params} -> RMSE: {rmse:.2f}")
                
                # Lógica del Campeón (Champion Logic)
                if rmse < best_score:
                    best_score = rmse
                    best_model_name = name
                    best_params = params
                    best_model_instance = model

    # --- Registro del Modelo Campeón ---
    print(f"\n>>> GANADOR DEFINITIVO: {best_model_name} con RMSE: {best_score:.2f} <<<")
    
    # Registramos al ganador en la corrida padre para fácil acceso
    mlflow.log_metric("champion_rmse", best_score)
    mlflow.set_tag("champion_model", best_model_name)
    mlflow.set_tag("model_role", "Champion")
    
    # Guardamos el modelo físico en MLflow
    mlflow.sklearn.log_model(best_model_instance, "champion_model_artifact")

>>> Iniciando Torneo de Modelos con MLflow <<<

--> Evaluando Familia: Ridge
    {'alpha': 0.1} -> RMSE: 15.03
🏃 View run Challenger_Ridge at: http://127.0.0.1:5000/#/experiments/608006539835666242/runs/53bd64360627445c9e908ad756619d37
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/608006539835666242
    {'alpha': 1.0} -> RMSE: 15.03
🏃 View run Challenger_Ridge at: http://127.0.0.1:5000/#/experiments/608006539835666242/runs/246171e2c40e496f942c66785308829e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/608006539835666242
    {'alpha': 10.0} -> RMSE: 15.03
🏃 View run Challenger_Ridge at: http://127.0.0.1:5000/#/experiments/608006539835666242/runs/8a7d3c521243491099535fd83155d595
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/608006539835666242

--> Evaluando Familia: Lasso
    {'alpha': 0.01} -> RMSE: 15.21
🏃 View run Challenger_Lasso at: http://127.0.0.1:5000/#/experiments/608006539835666242/runs/1d5b84a32c5841a0ba8ec4ccf78f510d
🧪 View experiment at: ht

2025/12/16 00:08:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


    {'learning_rate': 0.2} -> RMSE: 6.05
🏃 View run Challenger_GradientBoosting at: http://127.0.0.1:5000/#/experiments/608006539835666242/runs/c5b81e61c32544458a473023ab0c36cc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/608006539835666242

>>> GANADOR DEFINITIVO: RandomForest con RMSE: 5.46 <<<
🏃 View run Notebook_Training_Cycle at: http://127.0.0.1:5000/#/experiments/608006539835666242/runs/374fb27b09dc4e0db5c153b3ac7df25b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/608006539835666242


In [9]:
# Crear Pipeline Final con el Ganador
final_pipeline = Pipeline([
    ('preprocessor', feature_pipeline),
    ('model', best_model_instance)
])

# Guardar localmente
output_path = '../models/sales_forecasting_pipeline_v1.pkl'
joblib.dump(final_pipeline, output_path)
print(f"Pipeline local guardado en: {output_path}")
print("Nota: El modelo también ha sido guardado en el servidor de MLflow.")

Pipeline local guardado en: ../models/sales_forecasting_pipeline_v1.pkl
Nota: El modelo también ha sido guardado en el servidor de MLflow.
